# 사진 읽기

In [6]:
img = cv2.imread("sample.jpg")
cv2.imshow("img", img)
cv2.waitKey()
cv2.destroyAllWindows()

# 파일명 변경

In [23]:
import os

# 주어진 디렉토리에 있는 항목들의 이름을 담고 있는 리스트를 반환합니다.
# 리스트는 임의의 순서대로 나열됩니다.
file_path = 'C:\py_data\helmet_detection\dataset\helmet_orgin'
file_names = os.listdir(file_path)
len(file_names)

7260

In [24]:
i = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

# 중복된 사진 제거

In [3]:
photo_list = []

for f in os.listdir('C:\py_data\helmet_detection\dataset\helmet_orgin'):
    if 'jpg' or 'png' in f:
        photo_list.append(f)

In [4]:
photo_size = list(map(lambda x: os.path.getsize('C:\py_data\helmet_detection\dataset\helmet_orgin' + '/' + x), photo_list))

In [5]:
import pandas as pd

# Find Same Photos
fsp = pd.DataFrame({'filename':photo_list, 'size':photo_size})

print('사진의 갯수 :', len(fsp))

사진의 갯수 : 8177


In [6]:
fsp[:2]

,filename,size
0,helmet1.jpg,4520
1,helmet10.jpg,204986


In [7]:
# Photo Value Counts
pvc = pd.DataFrame({'filename':fsp['filename'].value_counts().index, 'fn_counts':fsp['filename'].value_counts().values})   
psvc = pd.DataFrame({'size':fsp['size'].value_counts().index, 'size_counts':fsp['size'].value_counts().values})   

fsp = pd.merge(fsp, pvc, how = 'left', on = 'filename')
fsp = pd.merge(fsp, psvc, how = 'left', on = 'size')

fsp.sample(2)

,filename,size,fn_counts,size_counts
7960,helmet8163.jpg,320852,1,1
519,helmet1466.jpg,351279,1,1


In [8]:
for i in range(len(fsp)):
    if (fsp['fn_counts'][i] > 1) & (fsp['size_counts'][i] == 1):
        print(i)

In [9]:
# Find Same Phto_Not Same Name
fsp_nsn = fsp.sort_values(['filename'], ascending = False).drop_duplicates(['filename'], keep = 'first')

In [10]:
print('남은 사진의 갯수 : {}\n지워진 사진의 갯수 : {}'.format(len(fsp_nsn), len(fsp)-len(fsp_nsn)))

남은 사진의 갯수 : 8177
지워진 사진의 갯수 : 0


In [11]:
pvc_nsn = pd.DataFrame({'filename':fsp_nsn['filename'].value_counts().index, 'fn_counts_nsn':fsp_nsn['filename'].value_counts().values})   
psvc_nsn = pd.DataFrame({'size':fsp_nsn['size'].value_counts().index, 'size_counts_nsn':fsp_nsn['size'].value_counts().values})   

fsp_nsn = pd.merge(fsp_nsn, pvc_nsn, how = 'left', on = 'filename')
fsp_nsn = pd.merge(fsp_nsn, psvc_nsn, how = 'left', on = 'size')

In [12]:
fsp_nsn[fsp_nsn['fn_counts_nsn']!=1]

,filename,size,fn_counts,size_counts,fn_counts_nsn,size_counts_nsn


In [13]:
print('사이즈 겹치는 사진의 갯수 :', len(fsp_nsn[fsp_nsn['size_counts_nsn']!=1]))
print('중복 사이즈의 갯수 :', len(psvc_nsn[psvc_nsn['size_counts_nsn']>1]))

사이즈 겹치는 사진의 갯수 : 3166
중복 사이즈의 갯수 : 1281


In [15]:
fsp_nsn[fsp_nsn['size_counts']!=1]

,filename,size,fn_counts,size_counts,fn_counts_nsn,size_counts_nsn
9,helmet990.jpg,8711,1,3,1,3
22,helmet979.jpg,8632,1,3,1,3
46,helmet957.jpg,7739,1,3,1,3
58,helmet946.jpg,17644,1,2,1,2
70,helmet935.jpg,33203,1,2,1,2
...,...,...,...,...,...,...
8147,helmet1024.jpg,7390,1,2,1,2
8152,helmet102.jpg,7978,1,2,1,2
8159,helmet1013.jpg,25217,1,2,1,2
8171,helmet1002.jpg,55803,1,2,1,2


In [22]:
psvc_nsn[psvc_nsn['size_counts_nsn'] >= 2]

,size,size_counts_nsn
0,7458,9
1,6751,8
2,6633,8
3,5499,8
4,6525,7
...,...,...
1276,7451,2
1277,7353,2
1278,93914,2
1279,57116,2


In [16]:
import cv2   # OpenCV
#from skimage.measure import compare_ssim
from skimage.metrics import structural_similarity as compare_ssim

# 삭제될 사진의 리스트
delete = []

try:
    for i in range(len(psvc_nsn)):

        # 중복된 크기(size)가 있는 경우
        if psvc_nsn['size_counts_nsn'][i] >= 2:

            # 그 크기에 해당하는 사진을 불러온다. 
            temp = fsp_nsn[fsp_nsn['size']==psvc_nsn['size'][i]].reset_index(drop = True).sort_values(['filename'])

            # 사진 읽기
            imageA = cv2.imread('C:\py_data\helmet_detection\dataset\helmet_orgin/'+temp['filename'][0])
            imageB = cv2.imread('C:\py_data\helmet_detection\dataset\helmet_orgin/'+temp['filename'][1])

            # 이미지를 grayscale로 변환
            grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

            # 이미지의 구조가 같다면 이미지 비교
            if len(grayA)==len(grayB):
                (score, diff) = compare_ssim(grayA, grayB, full=True)

                # 차이가 없다면 하나는 delete에 넣어주기
                if score == 1:
                    delete.append(temp['filename_raw'][1])

                # 구조가 같지만 차이가 존재한다면 직접 확인하기     
                else:
                    print('확인해보시오! : ', temp['filename_raw'][0], '/', temp['filename_raw'][1], f'(score : {score})')
except:
    print(i)
    pass

2


In [18]:
# 중복제거된 것들은 delete 리스트에 넣어주기
delete = delete + list(fsp[~fsp['filename'].isin(fsp_nsn['filename'])]['filename'])

print('삭제할 목록 :', len(delete))

삭제할 목록 : 0


In [20]:
# result : 처음(fsp)데이터에서 - delete를 제외한 것
result = list(fsp[~fsp['filename'].isin(delete)]['filename'])

print('남길 목록 : ', len(result))

남길 목록 :  8177


In [ ]:
import shutil

for i in result:
    shutil.move('Photo/'+i, '/Users/mizy/Desktop/mizy/Image/Result')
    
for i in delete:
    shutil.move('Photo/'+i, '/Users/mizy/Desktop/mizy/Image/Delete')    

# 사진 자르기

In [25]:
import cv2
import cvlib as cv

In [27]:
p = 1
for i in range(1,7261):    
    try:
        img = cv2.imread("C:/py_data/helmet_detection/dataset/helmet_orgin/"+str(i)+".jpg")
        h, w, c = img.shape
        if(c != 3):
            pass
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces, confidences = cv.detect_face(img)


        for (x, y, x2, y2), conf in zip(faces, confidences):
            if len(faces)==0 :
                pass
            else:
                #비율
                width = x2 - x
                height = y2 - y

                #변수지정
                cropX = int(x-(width*0.3))
                cropY = int(y-(height*0.6))
                cropX2 = int(x2+(width*0.3))
                cropY2 = int(y2+(height*0.2))


                #이미지영역 내부 인지
                if ((cropX > 0 and cropX < w) and (cropY > 0 and cropY < h) and (cropX2 > 0 and cropX2 < w) and (cropY2 > 0 and cropY2 < h)):
                    cropped = img[cropY:cropY2, cropX:cropX2]
                    if cropX2 - cropX >= 80 :
                        cv2.imwrite("C:/py_data/helmet_detection/dataset/helmet_crop/crop_"+str(i)+"_"+str(p)+".jpg", cropped)
                else :
                    pass

            p+=1
      
    except:
            pass